In [ ]:
def MIP_robust(bond_data, lambda_penalty):
    
    N = len(bond_data)  # Number of bonds based on the DataFrame length
    W_max = 1.0  # Maximum weight for the portfolio
    
    # Robust parameters 
    epsilon = 1.96 # 95% confidence interval
    T = 1

    # Calculate additional fields
    bond_data['DTS'] = bond_data['duration'] * bond_data['OAS']
    bond_data['transaction_cost'] = bond_data['ask_price'] - bond_data['bid_price']
    bond_data['OAS'] = pd.to_numeric(bond_data['OAS'], errors='coerce')
    expected_return = np.array(bond_data['expected_return'])

    # Create the model
    model = gp.Model("MIP")

    # Decision variables
    w = model.addVars(N, vtype=GRB.CONTINUOUS, lb=0, name="w")  # weights for bonds
    x = model.addVars(N, vtype=GRB.BINARY, name="x")  # binary selection of bonds
    
    # Robust variable 
    y = model.addVars(N, vtype=GRB.CONTINUOUS, lb=0, name="y")

    # Constrain the total weight
    model.addConstr(gp.quicksum(w[i] for i in range(N)) <= W_max, "MaxWeight")
    model.addConstr(gp.quicksum(w[i] for i in range(N)) == 1, "WeightSum")

    # Set the objective: 
    model.setObjective(
    gp.quicksum((expected_return[i] * w[i] 
                 - epsilon * (bond_data.iloc[i]['return_std_dev'] / np.sqrt(T)) * y[i]
                 - bond_data.iloc[i]['transaction_cost'] * x[i]) for i in range(N))  # return minus transaction cost
                 - lambda_penalty * gp.quicksum(bond_data.iloc[i]['OAS'] * w[i] for i in range(N))  # OAS risk term
                 - lambda_penalty * gp.quicksum(bond_data.iloc[i]['DTS'] * w[i] for i in range(N)),  # DTS risk term
    GRB.MAXIMIZE
    )
    
    # Robust constraints
    model.addConstrs((y[i] >= w[i] for i in range(N)), "abs_value_pos")
    model.addConstrs((y[i] >= -w[i] for i in range(N)), "abs_value_neg")

    # Deviation from benchmark
    deviation_limit = 0.3  # 10% deviation
    BenchmarkWeight = 0.01  # 1% in each bond

    for i in range(N):
        model.addConstr(w[i] >= BenchmarkWeight - deviation_limit*BenchmarkWeight)
        model.addConstr(w[i] <= BenchmarkWeight + deviation_limit*BenchmarkWeight)

    # # binary 
    # for i in range(N):
    #     model.addConstr(w[i] <= x[i], f"WeightSelection_{i}")

    # OAS Constraints
    weighted_OAS = gp.quicksum(bond_data.iloc[i]['OAS'] * w[i] for i in range(N))
    benchmark_OAS = sum(0.01 * bond_data.iloc[i]['OAS'] for i in range(N))  # 1% in each bond
    lower_bound = 0.9 * benchmark_OAS
    upper_bound = 1.1 * benchmark_OAS

    model.addConstr(weighted_OAS >= lower_bound, name="OAS_LowerBound")
    model.addConstr(weighted_OAS <= upper_bound, name="OAS_UpperBound")

    # Liquidity Constraint
    Liquidity = gp.quicksum(bond_data.iloc[i]['liquidity_score'] for i in range(N)) / N
    MinLiquidity = 0.9 * Liquidity
    model.addConstr(gp.quicksum(bond_data.iloc[i]['liquidity_score'] * w[i] for i in range(N)) >= MinLiquidity, "MinLiquidity")

    # Transaction Cost Constraints
    Benchmark_cost = gp.quicksum(bond_data.iloc[i]['transaction_cost'] for i in range(N)) / N
    lower_t_cost = 0.9 * Benchmark_cost
    upper_t_cost = 1.1 * Benchmark_cost

    model.addConstr(gp.quicksum(bond_data.iloc[i]['transaction_cost'] * x[i] for i in range(N)) >= lower_t_cost, "MinTCost")
    model.addConstr(gp.quicksum(bond_data.iloc[i]['transaction_cost'] * x[i] for i in range(N)) <= upper_t_cost, "MaxTCost")

    # Optimize the model
    model.optimize()

    if model.status == GRB.OPTIMAL:
        #print("Optimal solution found. List of all weights:")
        weights = [w[i].X for i in range(N)]  # Get the optimized weights for bonds
        return np.array(weights)

    if model.status == GRB.INFEASIBLE:
        print("The model is infeasible.")
        